---
# Iniciamos el ETL del primer archivo steam_games

Importando librerías a utilizar

In [1]:
import json
import re
import ast
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


## Cargamos y transformamos la data

In [10]:

# Extracción del JSON y generación del DataFrame
with open("../DataJSon/output_steam_games.json") as file:
    df = pd.DataFrame([json.loads(line) for line in file.readlines()])


In [3]:
df.shape

(120445, 13)

In [4]:
df.describe()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
count,24083,28852,32133,30085,32135,30068,31972,32133,31465,30758.00,32135,32133,28836
unique,8239,883,32094,30054,32135,3582,15395,32132,4649,162.00,2,32132,10992
top,Ubisoft,[Action],Soundtrack,Soundtrack,http://store.steampowered.com/app/761140/Lost_...,2012-10-16,"[Casual, Simulation]",http://steamcommunity.com/app/612880/reviews/?...,[Single-player],4.99,False,612880,Ubisoft - San Francisco
freq,385,1880,3,3,1,100,1292,2,2794,4278.00,30188,2,1259


Inspeccionamos la columnas

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


### Existen columnas que no se utilizan por lo tanto se eliminan con el fin de optimizar el tratamiento de los datos

In [11]:
games = df.drop(['url', 'reviews_url'], axis=1)
games

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Iniciamos a tratar los nulos y repetidos

## Funciones para ayudarme en la limpieza

In [12]:

def convert_to_float_or_zero(value):
    """
    Convierte el valor a float; si es un valor str o nulo, devuelve 0.0.
    
    Parameters:
        value (any): El valor a intentar convertir a float.
    
    Returns:
        float: El valor convertido a float o 0.0 si la conversión no es posible.
    """
    if value is None or isinstance(value, str):
        return 0.0
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0
def convertir_fecha(df, columna):
    '''
    Convierte los datos de una columna a tipo fecha en formato "año-mes-día".
    Elimina las filas que contienen palabras como 'soon' u otras similares.

    Args:
        df (DataFrame): DataFrame que contiene la columna.
        columna (str): Nombre de la columna a convertir.

    Returns:
        DataFrame: DataFrame con la columna convertida y filas eliminadas.
    '''
    # Crea una copia del DataFrame para evitar modificaciones inesperadas
    df = df.copy()

    # Itera sobre la columna
    for i, fecha in enumerate(df[columna]):
        if pd.notna(fecha):  # Verifica que no sea nulo
            try:
                # Convierte la fecha al formato "año-mes-día"
                fecha_convertida = pd.to_datetime(fecha, errors='raise').strftime('%Y-%m-%d')
                # Asigna la fecha convertida
                df.at[i, columna] = fecha_convertida
            except (ValueError, TypeError):
                # En caso de error o tipo incorrecto, elimina la fila
                df.drop(i, inplace=True)

    # Elimina filas que contienen palabras como 'soon'
    df = df[~df[columna].str.contains(r'\bsoon\b', case=False, na=False)]

    # Reinicia el índice después de eliminar filas
    df.reset_index(drop=True, inplace=True)

    return df
def extraer_anio(fecha): # --- > Esta funcion tiene la tarea de realizar una limpieza de los anios en la columna de release_date, returna solo el anio dejando de lado lo demas
    if '-' in fecha:  # Si el formato es 'YYYY-MM-DD'
        return fecha.split('-')[0]
    else:  # --- >  Si el formato es 'MMM YYYY'
        partes = fecha.split(' ')
        if len(partes) == 2:
            return partes[1]
    return fecha


In [13]:
games.dropna(thresh=9,inplace=True) #Elimino las filas que cumplen con la condicion de estar completamente sin datos (NaN), de lo contrario si contienen algun valor no se eliminan
games = games.drop(games.loc[games['id'].isna()].index) #Realice una exploracion en la columna id, y tome la desicion de eliminar las filas que no contienen valores, no tienen relevancia en el resto del proceso
games['id'] = games['id'].astype(int) # Realice una conversion en el tipo de dato de la columna id, de Float a Int
games = games.drop_duplicates(subset=['id']) #Comprobe que existen valores repetidos, verifique que todos sus valores sean identicos y luego procedi a realizar su eliminacion.
# Primero comprobe que la columna genres y tags, contienen los mismos valores, complete los faltantes de la columna tags con los valores de la columna genres
filas_vacias = games['tags'].isna() #guardo las filas con valores NaN en tags
games.loc[filas_vacias, 'tags'] = games.loc[filas_vacias, 'genres'] # asigno los valores de la columna genres dentro de la columna tags
# Relleno los valores de la columna publisher, con los valores de la columna developer, solo cuando publisher no tiene ningun valor.
filas_vacias = games['publisher'].isna()
games.loc[filas_vacias, 'publisher'] = games.loc[filas_vacias, 'developer']
# En este caso no elimino ninguna de las columnas, ya que algunos valores que son relevantes tambien son diferentes.
games.dropna(inplace=True) #Elimino todos los NaN
# Utilizo mi funcion extraer_anio guardada y especificada en el directorio funcines, archivo funciones.py
games['release_date'] = games['release_date'].apply(extraer_anio)
# Normalizo los ultimos datos de la columna release_date
games['release_date'] = games['release_date'].replace('SOON™',2019)
games['release_date'] = games['release_date'].replace('soon',2019)
games['release_date'] = games['release_date'].replace('SOON',2019)
# Convierto la columna id de valores str a int
games['id'] = games['id'].astype(int)
# Convierto la columna release_date de valores str a int
games['release_date'] = games['release_date'].astype(int)

In [14]:
# Elimino las filas que tienen menos de 9 valores no nulos
games.dropna(thresh=9, inplace=True)
# Elimino la columna 'app_name ya que 'title' contiene la misma información y sin faltantes
games.drop(columns='app_name', inplace=True)
# Elimino las filas que no tienen un valor en la columna 'id'
games = games.dropna(subset=['id'])
# Convierto la columna 'id' de float a int
games['id'] = games['id'].astype(int)
# Elimino duplicados basándome en la columna 'id'
games = games.drop_duplicates(subset=['id'])
# Completo los valores faltantes en 'tags' con los valores de 'genres'
games['tags'].fillna(games['genres'], inplace=True)
# Relleno los valores faltantes en 'publisher' con los valores de 'developer' solo si 'publisher' está vacío
games['publisher'].fillna(games['developer'], inplace=True)


In [15]:
# Normalizo los ultimos datos de la columna release_date
games['release_date'] = games['release_date'].replace('SOON™',2019)
games['release_date'] = games['release_date'].replace('soon',2019)
games['release_date'] = games['release_date'].replace('SOON',2019)


In [16]:
games.head()

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd


In [17]:
games.dropna(inplace=True)

In [18]:
games

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018,"[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich


In [19]:
games.isna().sum()

publisher       0
genres          0
title           0
release_date    0
tags            0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

## Transformación de los datos 

In [20]:
# conozco que tipos de datos tiene el dataset para saber cual debo modificar
tipos_de_datos=games.dtypes
tipos_de_datos

publisher       object
genres          object
title           object
release_date     int32
tags            object
specs           object
price           object
early_access    object
id               int32
developer       object
dtype: object

In [21]:
#Aplicamos una función en la columna price para cambiar el tipo de dato
games['price'] = games['price'].apply(convert_to_float_or_zero)

In [22]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27181 entries, 88310 to 120443
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     27181 non-null  object 
 1   genres        27181 non-null  object 
 2   title         27181 non-null  object 
 3   release_date  27181 non-null  int32  
 4   tags          27181 non-null  object 
 5   specs         27181 non-null  object 
 6   price         27181 non-null  float64
 7   early_access  27181 non-null  object 
 8   id            27181 non-null  int32  
 9   developer     27181 non-null  object 
dtypes: float64(1), int32(2), object(7)
memory usage: 2.1+ MB


Usamos un diccionario para examinar las columnas del archivo, después lo convertimos a dataframe. El motivo es que, ya que hay datos anidados, revisamos antes de proseguir con el ETL

In [23]:
# Genero un diccionario vacío para almacenar los resultados
tipo_data = {"columna": games.columns, "tipos_de_datos": games.applymap(type).nunique()}

# Creo el DataFrame a partir del diccionario
analisis = pd.DataFrame(tipo_data)
analisis

,columna,tipos_de_datos
publisher,publisher,1
genres,genres,1
title,title,1
release_date,release_date,1
tags,tags,1
specs,specs,1
price,price,1
early_access,early_access,1
id,id,1
developer,developer,1


Imprimimos una vez más el archivo para comprobar su utilidad

In [24]:
games.head(15)

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
88316,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018,"[Free to Play, Indie, Simulation, Sports]","[Single-player, Multi-player, Online Multi-Pla...",9.99,False,774276,Poppermost Productions
88317,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,2018,"[Free to Play, Indie, Simulation, Sports]","[Single-player, Multi-player, Online Multi-Pla...",18.99,False,774277,Poppermost Productions
88318,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,2018,"[Free to Play, Indie, Simulation, Sports]","[Single-player, Multi-player, Online Multi-Pla...",29.99,False,774278,Poppermost Productions
88322,Stegalosaurus Game Development,"[Action, Adventure, Casual, Indie, RPG]",Army of Tentacles: (Not) A Cthulhu Dating Sim:...,2018,"[Action, Adventure, RPG, Indie, Casual]","[Single-player, Steam Achievements]",10.99,False,770380,Stegalosaurus Game Development
88323,Copperpick Studio,"[Casual, Indie]",Beach Rules,2018,"[Casual, Indie, Pixel Graphics, Cute, 2D]",[Single-player],3.99,False,768880,Copperpick Studio


In [25]:
games.tail(15)

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
120422,2K,"[Action, Adventure]",Mafia,2002,"[Action, Adventure, Violent, Story Rich, Class...",[Single-player],14.99,False,40990,Illusion Softworks
120423,Topware Interactive,"[Action, Casual, Indie]",Chicken Shoot Gold,2003,"[Action, Casual, Indie, Shooter]","[Single-player, Multi-player, Steam Trading Ca...",6.99,False,259340,ToonTRAXX Studios
120424,Valve,[Action],Day of Defeat,2003,"[FPS, World War II, Multiplayer, Action, Shoot...","[Multi-player, Valve Anti-Cheat enabled]",4.99,False,30,Valve
120425,Spiderweb Software,"[Strategy, RPG, Indie]",Geneforge 2,2003,"[RPG, Indie, Strategy, Turn-Based Combat, Isom...",[Single-player],19.99,False,200980,Spiderweb Software
120426,"Epic Games, Inc.",[Action],Unreal Tournament 2004: Editor's Choice Edition,2004,"[FPS, Arena Shooter, Action, Classic, Multipla...","[Single-player, Multi-player, Steam Trading Ca...",14.99,False,13230,"Epic Games, Inc."
120427,Valve,[Action],Counter-Strike: Condition Zero,2004,"[Action, FPS, Shooter, Multiplayer, Singleplay...","[Single-player, Multi-player, Valve Anti-Cheat...",9.99,False,80,Valve
120432,Fred Ells,[Indie],Agent X: Equation Rider,2018,[Indie],[Single-player],3.99,False,767010,Fred Ells
120434,Phil Fortier,"[Adventure, Indie]",Snail Trek - Chapter 3: Lettuce Be,2018,"[Adventure, Indie, Retro, Point & Click, Pixel...","[Single-player, Steam Achievements]",0.99,False,761480,Phil Fortier
120436,OrtiGames/OrtiSoft,"[Casual, Indie]",Raining blocks,2018,"[Indie, Casual]","[Single-player, Shared/Split Screen, Steam Ach...",0.99,False,767590,"Oscar Ortigueira López,OrtiGames/OrtiSoft"
120437,INGAME,"[Indie, RPG]",Bravium,2018,"[Indie, RPG, Puzzle, Tower Defense]","[Single-player, Steam Achievements, Steam Lead...",14.99,False,747320,INGAME


### Transformación de la columna "genres"

#### Terminada la transformación de los datos numéricos, se trata la columna "genres"

In [26]:
games["genres"]

88310         [Action, Casual, Indie, Simulation, Strategy]
88311                  [Free to Play, Indie, RPG, Strategy]
88312     [Casual, Free to Play, Indie, Simulation, Sports]
88313                           [Action, Adventure, Casual]
88315                       [Action, Adventure, Simulation]
                                ...                        
120439                   [Action, Adventure, Casual, Indie]
120440                [Casual, Indie, Simulation, Strategy]
120441                            [Casual, Indie, Strategy]
120442                          [Indie, Racing, Simulation]
120443                                      [Casual, Indie]
Name: genres, Length: 27181, dtype: object

In [27]:
#con explode hago que la columna genres tome cada valor que tiene almacenado y lo devuelva en un campo nuevo pero conservando el mismo id
games = games.explode("genres")
games = games.dropna(subset=["genres"])
games

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,Action,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88310,Kotoshiro,Casual,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88310,Kotoshiro,Indie,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88310,Kotoshiro,Simulation,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
88310,Kotoshiro,Strategy,Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
...,...,...,...,...,...,...,...,...,...,...
120442,Laush Studio,Indie,Russian Roads,2018,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120442,Laush Studio,Racing,Russian Roads,2018,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120442,Laush Studio,Simulation,Russian Roads,2018,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,Casual,EXIT 2 - Directions,2017,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [28]:
#busco espacios vacios o sin datos 
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67484 entries, 88310 to 120443
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     67484 non-null  object 
 1   genres        67484 non-null  object 
 2   title         67484 non-null  object 
 3   release_date  67484 non-null  int32  
 4   tags          67484 non-null  object 
 5   specs         67484 non-null  object 
 6   price         67484 non-null  float64
 7   early_access  67484 non-null  object 
 8   id            67484 non-null  int32  
 9   developer     67484 non-null  object 
dtypes: float64(1), int32(2), object(7)
memory usage: 5.1+ MB


## Guardo el dataset como csv

In [24]:
save = "../DataSets/steam_games.csv"

games.to_csv(save, index=False, encoding="utf-8")

## Transformo csv a parquet para una optimización de los archivos

In [108]:
#Transformo el archivo csv a parquet
#Leo el archivo csv
games= pd.read_csv("../DataSets/steam_games.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "../DataSets/steam_games.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(games)
pq.write_table(table,output_file)